### EX 5.8 DataLoaderNSplitters

문서의 내용을 분할하여 많은 내용을 순차적으로 LLM에게 읽을 수있도록 하는 작업
문서 파일을 읽고, 문서의 내용을 어떤 기준으로 쪼갤지 정하면,문서의 내용이 일정한 규칙으로 분할된다.
이렇게 문서를 분할 하면 LLM이 문서를 찾을 때 필요한 조각만 기억하면 되기 때문에 효율적이다.(*많은 내용을 처리할수록 비용이 올라간다.)

해당 작업을 위한 도구들을 살펴보고, 문서 파일을 쪼개서 chatAI에게 학습시킨다.
문서에 대한 내용을 질의 하여 올바르게 응답하고 있는지 확인해보자.

* RetrievalQA를 사용하여 문서에 대한 RAG작업을 요청한다.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
# loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'Victory Mansions is a residential building where Winston Smith lives in the novel "1984." It is described as having a hallway that smells of boiled cabbage and old rag mats. The building has a lift that is usually not working, and the electric current is cut off during daylight hours as part of an economy drive for Hate Week. The flat where Winston lives is on the seventh floor and is characterized by a gritty and impoverished environment. The building is adorned with a colored poster depicting the face of a man with a caption that reads "BIG BROTHER IS WATCHING YOU."'